<a href="https://colab.research.google.com/github/LaisST/FIAP_202501_HandsOn_data_analytics/blob/main/BigData_Aula4_Operacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 3 - Aula 4
# Operações entre DataFrames e Armazenamento

## Configuração de Ambiente

In [1]:
# Importar biblioteca  do Python para manipular variáveis de ambiente e o sistema operacional.
import os

In [2]:
# instalar silenciosamente (-q) o pacote findspark, que ajuda o Python a localizar a instalação do Spark.
! pip install -q findspark

In [3]:
# Importar findspark
import findspark

In [4]:
# Instalar o Java JDK 8
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
# Baixar arquivo compacto Spark

! wget -q https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz

In [6]:
# Extrair o arquivo compactado do Spark (.tgz).
! tar xf spark-3.5.7-bin-hadoop3.tgz

In [7]:
# Remover o arquivo compactado depois da extração para economizar espaço.
! rm -rf spark-3.5.7-bin-hadoop3.tgz

In [8]:
# COnfigurar as variaveis de ambiente
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.5.7-bin-hadoop3'

In [10]:
# Inicializar o Spark e Criar uma sessão

findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('PySpark DataFrame').getOrCreate()


In [11]:
from pyspark import SparkContext

In [12]:
# Criar (ou recuperar) um SparkContext, que é a interface de baixo nível para o cluster Spark.
sc = SparkContext.getOrCreate()

## Transformação

São operações preguiçosas que criam um novo RDD a partir de outro.

Nada é realmente executado aqui ainda — o Spark apenas guarda o plano de execução.

### map()

In [28]:
data= [1, 2, 3, 4, 5] # Cria uma lista
myRDD= sc.parallelize(data) # Transforma a lista data em um RDD // Em resumo, ele distribui essa lista pelos nós do cluster (ou pelas partições do seu computador, se for local).
newRDD= myRDD.map(lambda x: x*2) # O método map() aplica a função lambda x: x * 2 a cada elemento do RDD.
print(newRDD.collect()) # O método collect() é uma ação que executa o pipeline de transformações e retorna os resultados para o driver (Python).

[2, 4, 6, 8, 10]


Resumindo

Cria uma lista [1, 2, 3, 4, 5];

Transforma em um RDD distribuído;

Multiplica cada número por 2;

Coleta e imprime o resultado.

### filter()

In [29]:
data= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # Cria uma lista de 1 a 10.
myRDD= sc.parallelize(data) # Transforma a lista em RDD
newRDD= myRDD.filter(lambda x: x%2 == 0) # O método filter() é uma transformação que mantém apenas os elementos que atendem à condição especificada
print(newRDD.collect())

[2, 4, 6, 8, 10]


Resumo
Cria uma lista de 1 a 10

Transforma em um RDD distribuido

Filtra apenas os números onde o resultado da divisão por 2 é 0 (números pares)

Coleta e imprime o resultado

### distinct()

In [31]:
data= [1, 1, 1, 2, 2, 2, 3, 3, 3, 3] # Cria uma lista com valores repetidos
myRDD= sc.parallelize(data) # Transforma a lista em um RDD
newRDD= myRDD.distinct() # O método distinct() remove os valores duplicados
print(newRDD.collect())

[2, 1, 3]


Resumo

Cria uma lista com valores repetidos

Transforma a lista em um RDD

Remove os valores duplicados

Coleta e imprime o resultado

### GroupByKey()

In [32]:
myRDD = sc.parallelize([('a', 1), ('a', 2), ('a', 3), ('b', 1)]) #Cria um RDD onde cada elemento é um par chave-valor
resultList= myRDD.groupByKey().mapValues(list) # O GroupByKey agrupa os valores por chave, mapValues() transforma os valores em uma lista python dentro do valor da chave
resultList.collect()

[('b', [1]), ('a', [1, 2, 3])]

Resumo

Cria um RDD com pares de chave e valor

Agrupa os valores pela chave

Converte os valores em uma lista (dentro das chaves correspondentes)

Executa e traz para o driver (coleta e imprime)

### ReduceByKey()

In [33]:
from operator import add # Importar operator
myRDD = sc.parallelize([('a', 1), ('a', 2), ('a', 3), ('b', 1)]) # Cria um RDD com pares de chave e valor
newRDD= myRDD.reduceByKey(add) #O método reduceByKey() combina os valores que têm a mesma chave usando uma função de redução. O (add) vem de operator e faz a soma entre dois valores.
newRDD.collect()

[('b', 1), ('a', 6)]

Resumo

Cria um RDD com pares de chave e valor

Soma os valores por chave

Executa e traz o resultado (coleta e imprime)

### SortByKey()

In [34]:
myRDD = sc.parallelize([('c', 1), ('d', 2), ('a', 3), ('b', 4)]) # Cria um RDD com pares de chave e valor
#sort by key
newRDD= myRDD.sortByKey() # O método sortByKey() ordena o RDD com base nas chaves, de forma crescente por padrão.
newRDD.collect()

[('a', 3), ('b', 4), ('c', 1), ('d', 2)]

Resumo

Cria um RDD com pares de chave e valor

Ordena o RDD com base nas chaves

Executa e retorna o resultado (Coleta e imprime)

### Union()

In [35]:
myRDD1 = sc.parallelize([1, 2, 3, 4]) # Cria um RDD
myRDD2 = sc.parallelize([ 3, 4, 5, 6, 7])# Cria outro RDD
#union of myRDD1 and myRDD2
newRDD = myRDD1.union(myRDD2) # O método union() combina os elementos dos dois RDDs em um único RDD.
newRDD.collect()

[1, 2, 3, 4, 3, 4, 5, 6, 7]

Resumo

Cria 2 RDDs a partir de listas

Une os dois Rdds

Remove duplicadas

Executa e retorna o resultado (Coleta e imprime)

## Ações

São as operações que disparam a execução real e retornam um resultado.

Quando uma ação é chamada, o Spark executa todo o pipeline de transformações necessário para gerar o resultado final.


### count()

In [36]:
data= [1, 1, 1, 4, 5, 6, 7, 8, 9, 10] # Cria uma lista com 10 elementos
myRDD= sc.parallelize(data) # Transforma essa lista em RDD
#Returns 4 as output
myRDD.count() #O método count() é uma ação do Spark que conta o número total de elementos (sem remover duplicatas).

10

### reduce()

In [38]:
data= [1, 2, 3, 4, 5] # Cria uma lista com 5 elementos
myRDD= sc.parallelize(data) # Transforma a lista em RDD
#returns the product of all the elements
myRDD.reduce(lambda x, y: x * y) #O método reduce() aplica a função binária (lambda x, y: x * y) que multiplica todos os elementos.

120

### foreach()

In [39]:
def fun(x):
    print(x) # Define a função que apenas imprime o valor que recebe no parametro
data= ['Scala', 'Python', 'Java', 'R'] # Cria uma lista de strings
myRDD= sc.parallelize(data) # Transforma a lista em RDD
#function applied to all the elements
myRDD.foreach(fun) #O método foreach() executa a função fun() em cada elemento do RDD, mas dentro das partições do cluster Spark, não no driver (seu programa principal).

### countByValue()

In [24]:
data= ['Python', 'Scala', 'Python', 'R', 'Python', 'Java', 'R']
myRDD= sc.parallelize(data)
#items() returns a list with all the dictionary keys and values returned by countByValue()
myRDD.countByValue().items()

dict_items([('Python', 3), ('Scala', 1), ('R', 2), ('Java', 1)])

In [40]:
#Continuação código anterior

for x in myRDD.collect():
    print(x)


Scala
Python
Java
R


Mesmo que você tenha usado print(x) dentro da função, é provável que nada apareça no console onde você executa o script

### countByKey()

In [25]:
data= [('a', 1), ('b', 1), ('c', 1), ('a', 1)] # Cria uma lista de pares de chave e valor
myRDD = sc.parallelize(data) # Transforma a lista em RDD
myRDD.countByKey().items() # Conta quantas vezes cada chave aparece no RDD

dict_items([('a', 2), ('b', 1), ('c', 1)])

### take(n)

In [41]:
data= [2, 5, 3, 8, 4] # Cria a lista com 5 elementos
myRDD= sc.parallelize(data) # Transforma a lista em RDD
#return the first 3 elements
myRDD.take(3) # Retorna os primeiros 3 elementos do RDD como uma lista Python.

[2, 5, 3]

### top(n)

In [42]:
data= [2, 5, 3, 8, 4] # Cria uma lista com 5 elementos
myRDD= sc.parallelize(data) # Transforma em RDD
#return the first 2 elements
myRDD.top(3) # Retorna os 3 maiores elementos do RDD, em ordem decrescente.

[8, 5, 4]

## Resumo da Aula

✅ **Transformações** → criam novos RDDs (como `map`, `filter`, `distinct`, `union`, `sortByKey`, etc.)

✅ **Ações** → executam o processamento e retornam resultados (como `collect`, `count`, `reduce`, `take`, `top`, `foreach`, `countByKey`, etc.)

Esses fundamentos são a base para evoluir depois para:

* 🔹 *RDDs avançados* (como `aggregateByKey` e `combineByKey`)
* 🔹 *Spark SQL e DataFrames* (para consultas mais otimizadas)
* 🔹 *Operações em clusters reais (compartilhamento, persistência e particionamento)*